#### Bond

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from datetime import datetime, timedelta
import time
from dateutil.relativedelta import relativedelta

In [ ]:
def get_accrued_interest(last_coupon_date, today, coupon_rate, amount, convention='30/360'):
    """
    Compute accrued interest 
    
    today, last_coupon_date: datetime 
    coupon_rate: in decimal, ie, 0.065 
    amount: notional 
    convention: options incl. "30/360", "actual/360", tbc 
    """
    coupon_rate = np.float(coupon_rate)
    
    if today < last_coupon_date: 
        print("date error")
        return np.nan 
    
    if convention == '30/360':
        d2, m2, y2 = today.day, today.month, today.year
        d1, m1, y1 = last_coupon_date.day, last_coupon_date.month, last_coupon_date.year 
        delta = (min(d2, 30) + max(0, 30-d1))/360 + (m2-m1-1)/12 + (y2-y1)   # in year 
    elif convention == 'actual/360':
        delta = (today - last_coupon_date).days / 360 
    return coupon_rate * delta * amount 




def get_datetime(date):
    if date == 'NaT':
        return datetime(100,1,1)
    y, m, d = date.split()[0].split('-')

    d = int(d)
    m = int(m)
    y = int(y)
    return datetime(y, m, d)




def get_month_by_freq(freq):
    if freq == 'semi-annual': 
        length = 6   # in months 
    elif freq == 'annual':
        length = 12
    elif freq == 'quarter':
        length = 3 
    return length 

In [ ]:
def get_coupon_payment_dates(first_int_accrual_date, first_cpn_date, maturity_date, freq='semi-annual'):
    """
    Inference all coupon payment dates according to an instrument's first interest accrual date / first_cpn_date & final maturity date 
    
    first_int_accrual_date, maturity_date, first_cpn_date: datetime
    freq: options incl. "quarter", "semi-annual", "annual", tbc 
    
    assume business day convention: Following (move payments to the next business day)
    ref: https://quant.stackexchange.com/questions/31506/how-to-compute-dates-for-bond
    """
    length = get_month_by_freq(freq)
    
    m2, y2 = maturity_date.month, maturity_date.year
    m1, y1 = first_cpn_date.month, first_cpn_date.year
    cnt = ((y2-y1)*12 + (m2-m1)) // length
    if first_int_accrual_date < first_cpn_date:
            dates = [first_int_accrual_date]
    else:
            dates = []
    
    dates.extend([first_cpn_date + relativedelta(months = (i+1)*length) for i in range(-1, cnt)]   )
    # i starts with -1 to include first_int_accrual_date

        
    # adjust for weekends 
    # 0:'Mon', 1:'Tue', 2:'Wed', 3:'Thu', 4:'Fri', 5:'Sat', 6:'Sun'
    for i in range(len(dates)):
        weekday = dates[i].weekday()
        if weekday == 5:
            dates[i] += relativedelta(days = 2)
        elif weekday == 6:
            dates[i] += relativedelta(days = 1)
    return dates


In [ ]:
class Cash_bond:
    def __init__(self, isin, name, ccy, exchange, sector, moody_rating, snp_rating, amount, midprice, reoffer, 
                 coupon_rate, tag, first_cpn_date, first_int_accrual_date, pricing_date, maturity, coupon_freq):
        """
        midprice, principal, accrued_interest: Series 
        """
        self.isin = isin
        self.name = name
        self.ccy = ccy
        self.exchange = exchange
        self.sector = sector
        self.moody_rating = moody_rating
        self.snp_rating = snp_rating
        self.amount = amount 
        
        self.reoffer = reoffer 
        self.coupon_rate = coupon_rate
        self.tag = tag 
        
        
        self.first_int_accrual_date = get_datetime(first_int_accrual_date)
        self.first_cpn_date = get_datetime(first_cpn_date)
        self.maturity_date = get_datetime(maturity)
        self.pricing_date = get_datetime(pricing_date)
            
        self.coupon_payment_freq = coupon_freq
        self.coupon_payment_dates = pd.Series(get_coupon_payment_dates(self.first_int_accrual_date, self.first_cpn_date, self.maturity_date, self.coupon_payment_freq))

        
        self.midprice = midprice.astype('float')
        prc_dt = self.pricing_date
        if prc_dt >= self.midprice.index[0] and prc_dt <= self.midprice.index[-1]:
            self.midprice.loc[:prc_dt] = 0   # no pricing data before and on first pricing date 
        elif prc_dt > self.midprice.index[-1]:
            self.midprice.loc[:] = 0   # bond hasn't come into existence  
            
            
        
    
    def principal(self):
        return self.amount * self.midprice / 100
    
    
    def accrued_interest(self, **kwargs):
        df = pd.Series().reindex_like(self.midprice).fillna(0) 
        for t in df.index:
            before = self.coupon_payment_dates[self.coupon_payment_dates <= t]
            if before.shape[0] > 0:
                last_coupon_date = before.iloc[-1]
                df.loc[t] = get_accrued_interest(last_coupon_date, t, self.coupon_rate, self.amount, convention='30/360')
        return df
    
    
    def dirty_price(self, **kwargs):
        return (self.accrued_interest(**kwargs) + self.principal()) 
    
    
    def coupon(self):
        df = pd.Series().reindex_like(self.midprice).fillna(0)
        frac = get_month_by_freq(self.coupon_payment_freq) / 12
        payment = self.coupon_rate * self.amount * frac    
        t = np.intersect1d(df.index, self.coupon_payment_dates)
        df.loc[t] = payment 
        
#         t_ = np.setdiff1d(self.coupon_payment_dates, t)
#         print(self.isin + " coupon payment missing " + ', '.join(t_.astype('str')))
        return df.cumsum().fillna(method='ffill')   # cumu
    
    
    def issuance(self):
        """
        Control for bond new issuance to adjust nav 
        """
        prc_dt = self.pricing_date
        df = pd.Series().reindex_like(self.midprice).fillna(0)
    
        if (prc_dt <= df.index.max()) and (prc_dt >= df.index.min())  :
            #   place new position on the next day of pricing day 
            find = df[df.index > prc_dt]
            if find.shape[0] > 0:
                prc_dt_ = find.index[0]
                
                df.loc[prc_dt_] = - self.reoffer / 100 * self.amount   # issuance price may not be 100 exactly  
                
        return df 
        
        

#### CDX

In [ ]:
class CDX(Cash_bond):   
    def __init__(self, isin, name, ccy, amount, midprice, 
                 coupon_rate, tag, first_cpn_date, first_int_accrual_date, maturity, coupon_freq, factor     ):
        """
        midprice, principal, accrued_interest, factor: Series 
        """
        exchange = ''; sector=''; moody_rating=''; snp_rating=''; tag=''; reoffer=0; pricing_date=first_int_accrual_date; 
        
        Cash_bond.__init__(self, isin, name, ccy, exchange, sector, moody_rating, snp_rating, amount, midprice, reoffer, 
                 coupon_rate, tag, first_cpn_date, first_int_accrual_date, pricing_date, maturity, coupon_freq)
        
        
        self.factor = factor   # % of remaining names in index without default 
        start = factor.dropna().index[0]
        if start > self.first_int_accrual_date:
            self.factor.loc[self.first_int_accrual_date: start] =1   # factor shall start from first accrual date and starts from 1  
        
        self.factor_ = self.factor.reindex(self.midprice.index).fillna(method='ffill')   # to fit the index of midprice 
        
        
        # assume T+5: turnover 5 days after the new CDX roll is released 
        self.roll_date_close = pd.bdate_range(self.first_int_accrual_date + relativedelta(months = 6) , periods = 6)[-1]   # periods = 6  as last series close date = next series open date    
        self.roll_date_open = pd.bdate_range(self.first_int_accrual_date, periods = 6)[-1]
        
        
    def accrued_interest(self, **kwargs):
        df = pd.Series().reindex_like(self.midprice).fillna(0)
        for t in df.index:
            before = self.coupon_payment_dates[self.coupon_payment_dates <= t]
            if before.shape[0] > 0:
                last_coupon_date = before.iloc[-1]  
                df.loc[t] = (-1) * get_accrued_interest(last_coupon_date, t, self.coupon_rate, self.amount, convention='actual/360') * self.factor_.loc[t]
        return df

    
    def principal(self):
        return ((100 - self.midprice) / 100 * self.factor_ * self.amount).fillna(0) 
    
    
    def npv(self, **kwargs):
        return (self.accrued_interest(**kwargs) + self.principal()).fillna(0)
    

    def exist(self):
        """
        Control for CDX rolling 
        """
        n_day = 5 
        start = self.first_int_accrual_date
        
        end = start + relativedelta(months = 6) 
        
        # change position 5 days after next cdx hy's pricing date   
        end = pd.bdate_range(end, periods = n_day)[-1]
        start = self.roll_date_open
        
        df = pd.Series().reindex_like(self.midprice).fillna(0)
        
        mask = np.logical_and(df.index >= start, df.index <= end)
        df.loc[mask] = 1 
        return df
    
    
    def roll_close(self):
        """
        Control for cash caused by rolling
        - Close position 
        """
        roll_date = self.roll_date_close
        df = pd.Series().reindex_like(self.midprice).fillna(0)
        if roll_date >= df.index.min() and roll_date <= df.index.max():
            df.loc[roll_date] = self.npv().loc[roll_date]
        return df 
    
    
    def roll_open(self):
        """
        Control for cash caused by rolling
        - Open position 
        """
        roll_date = self.roll_date_open
        df = pd.Series().reindex_like(self.midprice).fillna(0)
        if roll_date >= df.index.min() and roll_date <= df.index.max():
            df.loc[roll_date] = (-1) * self.npv().loc[roll_date]
        return df 
    

   